In [0]:
run Lattice_functions.ipynb

# Model 2: Constant Stiffness Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 20*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 4e4
nz = 3
lz = 0.01
k_vertical = 8e6
M2 = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M2 = add_braces_to_rectangular_lattice(M2, k_inPlane, '|')
M2 = add_layers_in_z_direction(M2, nz, lz, k_vertical, 0, [])

#setting z of the middle layer to zero and deriving support conditions:
min_x = M2.nodes[1].x
min_y = M2.nodes[1].y
max_x = M2.nodes[1].x
max_y = M2.nodes[1].y

for i in M2.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M2.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M2.support_x.append(i.n)
            M2.support_y.append(i.n)
            M2.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M2.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M2.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M2.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M2.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half unit force
f0 = Point_load_at_centre(0.5, M2)


#Variable for saving the data
data_M2 = [] #has the coordinates of the energy minimized lattices
forces_M2 = [] #forces applied at the centre
energies_M2 = [] #energies of the optimized lattices
z_centre_M2 = [] #deflection at centre
results_M2 = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
  print(i)
  f = (i - 1)*f0
  result = minimize(Energy, coord0, args = (f, M2), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M2.append(res)
  forces_M2.append(max(f))
  energies_M2.append(result.fun)
  z_centre_M2.append(res[2*nNodes + mid_node - 1])
  results_M2.append(result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [0]:
from numpy import save
save('data_M2.npy', data_M2)
save('forces_M2.npy', forces_M2)
save('energies_M2.npy', energies_M2)
save('z_centre_M2.npy', z_centre_M2)
save('results_M2.npy', results_M2)
save('M2.npy', M2)

# Model 2: Constant AE Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 20*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 10
nz = 3
lz = 0.01
k_vertical = 10

M2_AE = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M2_AE = add_braces_to_rectangular_lattice(M2_AE, k_inPlane, '|')
M2_AE = add_layers_in_z_direction(M2_AE, nz, lz, k_vertical, 0, [])

# modifying stiffness wrt AE = constant
AE = 1.28e3
for i in M2_AE.springs:
    i.k = AE/(i.l0)

for i in M2_AE.springs:
  if (abs(M2_AE.nodes[i.i - 1].z - M2_AE.nodes[i.j - 1].z) != 0):
    i.k = i.k*1e2


#setting z of the middle layer to zero and deriving support criterias:
min_x = M2_AE.nodes[1].x
min_y = M2_AE.nodes[1].y
max_x = M2_AE.nodes[1].x
max_y = M2_AE.nodes[1].y

for i in M2_AE.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M2_AE.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M2_AE.support_x.append(i.n)
            M2_AE.support_y.append(i.n)
            M2_AE.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M2_AE.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M2_AE.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M2_AE.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M2_AE.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half unit forces
f0 = Point_load_at_centre(0.5, M2_AE)

#Variable for saving the data
data_M2_AE = [] #has the coordinates of the energy minimized lattices
forces_M2_AE = [] #forces applied at the centre
energies_M2_AE = [] #energies of the optimized lattices
z_centre_M2_AE = [] #deflection at centre
results_M2_AE = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
    print(i)
    f = (i - 1)*f0
    result = minimize(Energy, coord0, args = (f, M2_AE), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
    res = result.x
    res.tolist()
    data_M2_AE.append(res)
    forces_M2_AE.append(max(f))
    energies_M2_AE.append(result.fun)
    z_centre_M2_AE.append(res[2*nNodes + mid_node - 1])
    results_M2_AE.append(result)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [0]:
from numpy import save
save('data_M2_AE.npy', data_M2_AE)
save('forces_M2_AE.npy', forces_M2_AE)
save('energies_M2_AE.npy', energies_M2_AE)
save('z_centre_M2_AE.npy', z_centre_M2_AE)
save('results_M2_AE.npy', results_M2_AE)
save('M2_AE.npy', M2_AE)

# Model 2: Constant Stiffness Model : Thickness variation

In [0]:
#Variable for saving the data
data_M2_t = [] #has the coordinates of the energy minimized lattices
thickness_M2_t = [] #Thickness of the model
energies_M2_t = [] #energies of the optimized lattices
z_centre_M2_t = [] #deflection at centre
results_M2_t = [] #Outputs from scipy.optimize
lat_M2_t = [] #Lattices with varying thickness

In [0]:
#Model 2, thickness varying condition
for i in range(1, 11):
  print(i)
  thickness = 0.005*i

  #Generating the lattice
  nx = 20 + 1 # 1 added to make the total length = 20*0.01
  ny = 20 + 1
  lx = 0.05
  ly = 0.05
  k_inPlane = 8e5
  nz = 3
  lz = thickness
  k_vertical = 8e5

  M2_t = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
  M2_t = add_braces_to_rectangular_lattice(M2_t, k_inPlane, '|')
  M2_t = add_layers_in_z_direction(M2_t, nz, lz, k_vertical, 0, [])

  #setting z of the middle layer to zero:
  min_x = M2_t.nodes[1].x
  min_y = M2_t.nodes[1].y
  max_x = M2_t.nodes[1].x
  max_y = M2_t.nodes[1].y

  for i in M2_t.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

  #Adding support
  for i in M2_t.nodes:
    if(i.z == 0):
      if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
        i.s_x = 1
        i.s_y = 1
        i.s_z = 1
        M2_t.support_x.append(i.n)
        M2_t.support_y.append(i.n)
        M2_t.support_z.append(i.n)


  mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny


  #Optimization preparation:
  x0 = []
  y0 = []
  z0 = []
  for i in M2_t.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
  coord0 = x0 + y0 + z0
  nNodes = int(len(coord0)/3)

  #bounds:
  lb = -np.inf*np.ones(len(coord0))
  ub = np.inf*np.ones(len(coord0))

  for i in M2_t.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

  for i in M2_t.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
  for i in M2_t.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


  bounds = Bounds(lb, ub)

  #Defining 2kN forces
  f = Point_load_at_centre(2, M2_t)

  result = minimize(Energy, coord0, args = (f, M2_t), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M2_t.append(res)
  thickness_M2_t.append(thickness)
  energies_M2_t.append(result.fun)
  z_centre_M2_t.append(res[2*nNodes + mid_node - 1])
  results_M2_t.append(result)
  lat_M2_t.append(M2_t)

In [0]:
from numpy import save
save('data_M2_t.npy', data_M2_t)
save('thickness_M2_t.npy', thickness_M2_t)
save('energies_M2_t.npy', energies_M2_t)
save('z_centre_M2_t.npy', z_centre_M2_t)
save('results_M2_t.npy', results_M2_t)
save('lat_M2_t.npy', lat_M2_t)